In [1]:
from dgp import *

# 1. Simulate or load data
sample_size = 800
min_degree = 8
max_degree = 10
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=50,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

100%|██████████| 70/70 [00:02<00:00, 30.53it/s]


In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.14641875000000001,
 'direct_effect': -0.10839374999999998,
 'spillover_effect': -0.45620625000000004,
 'psi_1_gamma': 0.11685625,
 'psi_0_gamma': 0.22524999999999998,
 'psi_zero': 0.68145625}

In [3]:
n_cpu = 10

In [4]:
from utils import run_pll
from run_pll import run_dr

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_dr)

Multiprocessing <function run_dr at 0x10ad2e4d0> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [5]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.12815 ± 0.04528
direct_effect: -0.18172 ± 0.39998
spillover_effect: -0.14737 ± 0.47227
psi_0_gamma: 0.27744 ± 0.38970
psi_zero: 0.42481 ± 0.18862
psi_1_gamma: 0.09572 ± 0.04397


In [6]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)
# save results
np.save(f'run/sim_results/sim_autog_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_ag)

Multiprocessing <function run_autognet at 0x3121949d0> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [7]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.13323 ± 0.04233
direct_effect: -0.04670 ± 0.04223
spillover_effect: -0.23182 ± 0.18608
psi_0_gamma: 0.16830 ± 0.04223
psi_zero: 0.40012 ± 0.21598
psi_1_gamma: 0.12160 ± 0.04944


In [8]:
from utils import run_pll
from run_pll import run_ocnet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_ocnet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)
# save results
np.save(f'run/sim_results/sim_ocnet_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_ag)

Multiprocessing <function run_ocnet at 0x312194a60> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [9]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.13574 ± 0.03651
direct_effect: -0.05751 ± 0.04613
spillover_effect: -0.24883 ± 0.15720
psi_0_gamma: 0.17598 ± 0.05738
psi_zero: 0.42481 ± 0.18862
psi_1_gamma: 0.11847 ± 0.03363
